<a href="https://colab.research.google.com/github/guumaster/colab-nbs/blob/master/fastai/exercises/pytorch_mnist_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor

train = datasets.MNIST(root="sample_data", download=True, train=True, transform=ToTensor())

dataset = DataLoader(train, 32, shuffle=True)


In [8]:
class ImageClassifier(nn.Module):
  def __init__(self):
      super(ImageClassifier, self).__init__()
      self.model = nn.Sequential(
          nn.Conv2d(1, 32, (3,3)),
          nn.ReLU(),
          nn.Conv2d(32, 64, (3,3)),
          nn.ReLU(),
          nn.Conv2d(64, 64, (3,3)),
          nn.ReLU(),
          nn.Flatten(),
          # NOTE: 28-6 related to removed pixels on previous layers. Why?
          nn.Linear(64*(28-6)*(28-6), 10)
      )

  def forward(self, x):
    return self.model(x)


In [9]:
clf = ImageClassifier().to('cuda')

opt = Adam(clf.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()


In [ ]:
for epoch in range(10):
  for batch in dataset:
    X, y = batch
    X, y = X.to('cuda'), y.to('cuda')
    yhat = clf(X)
    loss = loss_fn(yhat, y)

    opt.zero_grad()
    loss.backward()
    opt.step()

  print(f"Epoch {epoch} loss is {loss.item()}")

with open('model.pt', 'wb') as f:
  save(clf.state_dict(), f)



In [20]:
import requests 

img_url = "https://raw.githubusercontent.com/nicknochnack/PyTorchin15/main/img_3.jpg"
img = Image.open(requests.get(img_url, stream = True).raw)
img

In [21]:
import torch
from PIL import Image

with open('model.pt', 'rb') as f:
  clf.load_state_dict(load(f))
  img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

  print(torch.argmax(clf(img_tensor)))
  

tensor(9, device='cuda:0')
